In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sympy import diff
from sympy import symbols
import sympy as sp
import matplotlib.pyplot as plt

#   teste novo 

In [2]:
def demanda(X1,X2,X3,X4,X5,X6,P):
    D = np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 -0.0983*X4 + 0.3867*X5 -0.6541*X6 -0.0003*P  ) / (1 + np.exp(np.exp(0.1107 + 0.0275*X1+0.5022*X2+0.5601*X3-0.0983*X4+0.3867*X5-0.6541*X6 -0.0003*P  ) ))
    return D

In [3]:
df = pd.DataFrame({
    'X1': [10,12,15,17,19,10,20,10,15,20],
    'X2':[0,0,0,0,0,0,0,0,0,0],
    'X3':[0,0,0,0,0,0,0,0,0,1],
    'X4':[0,1,0,1,0,0,0,0,0,0],
    'X5':[0,0,0,0,0,0,0,0,0,0],
    'X6':[1,0,0,0,0,0,0,0,0,0],
    'P' : [3545.970152,5238.504747,3910.459,2228.152045,2330.735538,2800.23,1199.590955,1456.465285,5367.866686,2253.42037],
    'custo' :[2216.585942,3169.819222,596.641742,1236.8472,1247.176587,867.223121,629.905211,910.43645,3087.060131,1183.271036]
})
df['D']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['P'])
df

,X1,X2,X3,X4,X5,X6,P,custo,D
0,10,0,0,0,0,1,3545.970152,2216.585942,0.114639
1,12,0,0,1,0,0,5238.504747,3169.819222,0.125027
2,15,0,0,0,0,0,3910.459000,596.641742,0.194404
3,17,0,0,1,0,0,2228.152045,1236.847200,0.251789
4,19,0,0,0,0,0,2330.735538,1247.176587,0.263690
5,10,0,0,0,0,0,2800.230000,867.223121,0.219919
6,20,0,0,0,0,0,1199.590955,629.905211,0.277911
7,10,0,0,0,0,0,1456.465285,910.436450,0.264945
8,15,0,0,0,0,0,5367.866686,3087.060131,0.140431
9,20,0,1,0,0,0,2253.420370,1183.271036,0.260924


In [4]:
df['D'].sum()

2.113678785018064

In [5]:
df['D'].mean()

0.2113678785018064

In [7]:
# Exemplo de uso
lista_variaveis_na_ordem = ['X1','X2','X3','X4','X5','X6']
indicadora=np.array(df.loc[:,lista_variaveis_na_ordem])                                             
precos_iniciais = np.array(df['P'])*1
target =df['D'].sum()
custo=df['custo'].mean()

In [8]:
import numpy as np
import sympy as sp
from scipy.optimize import fsolve
import scipy.optimize as optimize

# Define a função de demanda
def demanda(X1,X2,X3,X4,X5,X6,P):
    D = np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 -0.0983*X4 + 0.3867*X5 -0.6541*X6 -0.0003*P  ) / (1 + np.exp(np.exp(0.1107 + 0.0275*X1+0.5022*X2+0.5601*X3-0.0983*X4+0.3867*X5-0.6541*X6 -0.0003*P  ) ))
    return D

# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i,0],
                      indicadora[i,1],
                      indicadora[i,2],
                      indicadora[i,3],
                      indicadora[i,4],
                      indicadora[i,5],
                      P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target)

# Aplicando a diferenciação numérica
def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

# Definindo a função para otimizar os preços
def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Calculando os preços otimizados
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora),xtol=1e-06, maxfev=1000)
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = X_otimizado[:-1]
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
print("Preços Otimizados:",precos_otimizados)

λ: -4800.414348139431
Preços Otimizados: [-4252.55275222  2204.24793525  2556.31643772  2456.29105995
  2796.8482265   2239.28199532 -2795.99202319  2184.91069059
  2497.87076654 -1715.94747997]


C:\ProgramData\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [9]:
df2=df
df2["precos_otimizados"]=precos_otimizados
df2['D*']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['precos_otimizados'])
df2

,X1,X2,X3,X4,X5,X6,P,custo,D,precos_otimizados,D*
0,10,0,0,0,0,1,3545.970152,2216.585942,0.114639,-4252.552752,0.166347
1,12,0,0,1,0,0,5238.504747,3169.819222,0.125027,2204.247935,0.236890
2,15,0,0,0,0,0,3910.459000,596.641742,0.194404,2556.316438,0.245712
3,17,0,0,1,0,0,2228.152045,1236.847200,0.251789,2456.291060,0.244193
4,19,0,0,0,0,0,2330.735538,1247.176587,0.263690,2796.848226,0.249923
5,10,0,0,0,0,0,2800.230000,867.223121,0.219919,2239.281995,0.240804
6,20,0,0,0,0,0,1199.590955,629.905211,0.277911,-2795.992023,0.050226
7,10,0,0,0,0,0,1456.465285,910.436450,0.264945,2184.910691,0.242715
8,15,0,0,0,0,0,5367.866686,3087.060131,0.140431,2497.870767,0.247685
9,20,0,1,0,0,0,2253.420370,1183.271036,0.260924,-1715.947480,0.019446


In [ ]:
print(df2['D'].mean()),
print(df2['D'].sum()),
print(df2['D*'].mean())

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize


df = pd.DataFrame({
    'X1': [10,12,15,17,19,10,20,10,15,20],
    'X2':[0,0,0,0,0,0,0,0,0,0],
    'X3':[0,0,0,0,0,0,0,0,0,1],
    'X4':[0,1,0,1,0,0,0,0,0,0],
    'X5':[0,0,0,0,0,0,0,0,0,0],
    'X6':[1,0,0,0,0,0,0,0,0,0],
    'P' : [3545.970152,5238.504747,3910.459,2228.152045,2330.735538,2800.23,1199.590955,1456.465285,5367.866686,2253.42037],
    'custo' :[2216.585942,3169.819222,596.641742,1236.8472,1247.176587,867.223121,629.905211,910.43645,3087.060131,1183.271036]
})
df['D']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['P'])
df


# Definir a função de demanda
def demanda(X1, X2, X3, X4, X5, X6, P):
    D = np.exp(0.1107 + 0.0275 * X1 + 0.5022 * X2 + 0.5601 * X3 - 0.0983 * X4 + 0.3867 * X5 - 0.6541 * X6 - 0.0003 * P)
    return D / (1 + D)


df['D'] = demanda(df['X1'], df['X2'], df['X3'], df['X4'], df['X5'], df['X6'], df['P'])

lista_variaveis_na_ordem = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6']
indicadora = np.array(df.loc[:, lista_variaveis_na_ordem])
precos_iniciais = np.array(df['P'])
target = df['D'].sum()
custo = df['custo'].mean()

# Função objetivo (com restrições)
def func(X, indicadora, precos_iniciais):
    P = np.maximum(X[:-1], 0.9 * precos_iniciais*1.1)  # Garante que precos_otimizados >= 0.9 * precos_iniciais
    L = X[-1]
    n = len(P)
    
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5], P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i

    # Função de Lagrange com a restrição da demanda otimizada igual ao target
    return objetivo - L * (soma_demandas - target)

# Diferenciação numérica
def dfunc(X, indicadora, precos_iniciais):
    dLambda = np.zeros(len(X))
    h = 1e-3
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora, precos_iniciais) - func(X - dX, indicadora, precos_iniciais)) / (2 * h)
    return dLambda

# Otimizar preços
def otimizar_precos(indicadora, precos_iniciais):
    X_inicial = np.append(precos_iniciais, 0)  # Valor inicial para o multiplicador de Lagrange
    
    # Otimização utilizando fsolve
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora, precos_iniciais), xtol=1e-06, maxfev=1000)
    
    precos_otimizados = np.maximum(X_otimizado[:-1], 0.9 * precos_iniciais*1.1)  # Garantir limite inferior de 0.9
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

# Executando a otimização
precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
print("Preços Otimizados:", precos_otimizados)



In [ ]:
df2=df
df2["precos_otimizados"]=precos_otimizados
df2['D*']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['precos_otimizados'])
df2

In [ ]:
print(df2['D'].mean()),
print(df2['D*'].mean())

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize


df = pd.DataFrame({
    'X1': [10,12,15,17,19,10,20,10,15,20],
    'X2':[0,0,0,0,0,0,0,0,0,0],
    'X3':[0,0,0,0,0,0,0,0,0,1],
    'X4':[0,1,0,1,0,0,0,0,0,0],
    'X5':[0,0,0,0,0,0,0,0,0,0],
    'X6':[1,0,0,0,0,0,0,0,0,0],
    'P' : [3545.970152,5238.504747,3910.459,2228.152045,2330.735538,2800.23,1199.590955,1456.465285,5367.866686,2253.42037],
    'custo' :[2216.585942,3169.819222,596.641742,1236.8472,1247.176587,867.223121,629.905211,910.43645,3087.060131,1183.271036]
})
df['D']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['P'])
df


# Definir a função de demanda
def demanda(X1, X2, X3, X4, X5, X6, P):
    D = np.exp(0.1107 + 0.0275 * X1 + 0.5022 * X2 + 0.5601 * X3 - 0.0983 * X4 + 0.3867 * X5 - 0.6541 * X6 - 0.0003 * P)
    return D / (1 + D)


df['D'] = demanda(df['X1'], df['X2'], df['X3'], df['X4'], df['X5'], df['X6'], df['P'])

lista_variaveis_na_ordem = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6']
indicadora = np.array(df.loc[:, lista_variaveis_na_ordem])
precos_iniciais = np.array(df['P'])
tarifa = np.array(df['P'])
target = df['D'].sum()
custo = df['custo'].mean()

# Função objetivo (com restrições)
def func(X, indicadora, tarifa):
    P = np.maximum(X[:-1], 0.9 * tarifa*1.1)  # Garante que precos_otimizados >= 0.9 * precos_iniciais
    L = X[-1]
    n = len(P)
    
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5], P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i

    # Função de Lagrange com a restrição da demanda otimizada igual ao target
    return objetivo - L * (soma_demandas - target)

# Diferenciação numérica
def dfunc(X, indicadora, tarifa):
    dLambda = np.zeros(len(X))
    h = 1e-3
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora, precos_iniciais) - func(X - dX, indicadora, precos_iniciais)) / (2 * h)
    return dLambda

# Otimizar preços
def otimizar_precos(indicadora, precos_iniciais):
    X_inicial = np.append(precos_iniciais, 0)  # Valor inicial para o multiplicador de Lagrange
    
    # Otimização utilizando fsolve
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora, precos_iniciais), xtol=1e-06, maxfev=1000)
    
    precos_otimizados = np.maximum(X_otimizado[:-1], precos_iniciais*1.01)  # Garantir limite inferior de 0.9
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

# Executando a otimização
precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
print("Preços Otimizados:", precos_otimizados)



NameError: name 'demanda' is not defined

In [ ]:
df2=df
df2["precos_otimizados"]=precos_otimizados
df2['D*']=demanda(df['X1'],df['X2'],df['X3'],df['X4'],df['X5'],df['X6'],df['precos_otimizados'])
df2

In [ ]:
print(df2['D'].mean()),
print(df2['D*'].mean())

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize

def demanda(X1, X2, X3, X4, X5, X6, P):
    D = np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 - 0.0983*X4 + 0.3867*X5 - 0.6541*X6 - 0.0003*P) / (1 + np.exp(np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 - 0.0983*X4 + 0.3867*X5 - 0.6541*X6 - 0.0003*P)))
    return D

df = pd.DataFrame({
    'X1': [10, 12, 15, 17, 19, 10, 20, 10, 15, 20],
    'X2': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'X3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    'X4': [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    'X5': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'X6': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'P': [3545.970152, 5238.504747, 3910.459, 2228.152045, 2330.735538, 2800.23, 1199.590955, 1456.465285, 5367.866686, 2253.42037],
    'custo': [2216.585942, 3169.819222, 596.641742, 1236.8472, 1247.176587, 867.223121, 629.905211, 910.43645, 3087.060131, 1183.271036]
})

df['D'] = demanda(df['X1'], df['X2'], df['X3'], df['X4'], df['X5'], df['X6'], df['P'])

lista_variaveis_na_ordem = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6']
indicadora = np.array(df.loc[:, lista_variaveis_na_ordem])
precos_iniciais = np.array(df['P']) * 1
target = df['D'].sum()
custo = df['custo'].mean()

def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5], P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target)

def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Definindo as restrições para garantir que os preços não sejam negativos
    bounds = [(0, None) for _ in range(len(precos_iniciais))] + [(None, None)]
    
    # Calculando os preços otimizados
    resultado = optimize.minimize(lambda X: np.sum(dfunc(X, indicadora)**2), X_inicial, bounds=bounds, method='L-BFGS-B')
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = resultado.x[:-1]
    lambida = resultado.x[-1]
    
    return precos_otimizados, lambida

precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
print("Preços Otimizados:", precos_otimizados)

In [ ]:
df['D'].mean()

In [ ]:
# Dados de exemplo
df = pd.DataFrame({
    'X1': np.random.randint(10, 20, size=50000),
    'X2': np.random.randint(0, 2, size=50000),
    'X3': np.random.randint(0, 2, size=50000),
    'X4': np.random.randint(0, 2, size=50000),
    'X5': np.random.randint(0, 2, size=50000),
    'X6': np.random.randint(0, 2, size=50000),
    'P' : np.random.uniform(1000, 6000, size=50000),
    'custo' : np.random.uniform(500, 3000, size=50000)
})


In [ ]:
import numpy as np 
import pandas as pd 
from scipy.optimize import minimize 
import multiprocessing as mp 

 
# Função de demanda 
def demanda(X1, X2, X3, X4, X5, X6, P): 
    D = np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 - 0.0983*X4 + 0.3867*X5 - 0.6541*X6 - 0.0003*P) 
    return D / (1 + D) 
# Função de Lagrange 
def func(P, indicadora, custo, target, n): 
    soma_demandas = 0 
    objetivo = 0 
    for i in range(n): 
        D_i = demanda(indicadora[i, 0], 
                      indicadora[i, 1],
                      indicadora[i, 2], 
                      indicadora[i, 3], 
                      indicadora[i, 4], 
                      indicadora[i, 5], P[i]) 
        objetivo += D_i * (P[i] - custo) 
        soma_demandas += D_i 
        return - (objetivo - 100 * abs(soma_demandas - target*df.shape[0])) 
# Função para otimizar preços em paralelo 
def otimizar_precos_em_paralelo(indicadora, precos_iniciais, custo, target):
    n = len(precos_iniciais) 
    bounds = [(0, None) for _ in range(n)]         # Garantir que os preços sejam positivos 
    with mp.Pool(mp.cpu_count()) as pool:
        result = pool.apply_async(minimize, (func, precos_iniciais, (indicadora, custo, target, n), 'L-BFGS-B', bounds)) 
        resultado = result.get() 
            
    precos_otimizados = np.clip(resultado.x, 0.9 * precos_iniciais, 1.1 * precos_iniciais)      
    # Restringir os preços entre 0.9x e 1.1x dos preços iniciais 
    return precos_otimizados, resultado.success 

# Gerar exemplo expandido com 1000 linhas 
np.random.seed(42) 
df = pd.DataFrame({'X1': np.random.randint(10, 20, 50), 
                                        'X2': np.random.randint(0, 2, 50), 
                                        'X3': np.random.randint(0, 2, 50), 
                                        'X4': np.random.randint(0, 2, 50), 
                                        'X5': np.random.randint(0, 2, 50), 
                                        'X6': np.random.randint(0, 2, 50), 
                                        'P': np.random.uniform(1000, 6000, 50), 
                                        'custo': np.random.uniform(500, 3000, 50) }) 
            
df['D'] = demanda(df['X1'], df['X2'], df['X3'], df['X4'], df['X5'], df['X6'], df['P']) 
            
lista_variaveis_na_ordem = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6'] 
            
indicadora = np.array(df.loc[:, lista_variaveis_na_ordem]) 
            
precos_iniciais = np.array(df['P']) 
target = 0.45
custo = df['custo'].mean() 
            
# Executar otimização em paralelo 
precos_otimizados, sucesso = otimizar_precos_em_paralelo(indicadora, precos_iniciais, custo, target) 
print("Otimização bem-sucedida:", sucesso) 
print("Primeiros 10 preços otimizados:", precos_otimizados[:10])

In [ ]:
df['D'].mean()